In [349]:
import geopandas as gpd 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [350]:
# load the data
dataset = gpd.read_file('./data.gpkg')
print('dataset size: ', dataset.shape[0])
print('dataset columns: ', dataset.columns.values)

dataset size:  1456
dataset columns:  ['TxID' 'StrNr' 'Strak' 'BdlNr' 'Bandel' 'Forbind_1' 'PlSignFr' 'PlNamnFr'
 'PlSignTi' 'PlNamnTi' 'AntalSpar' 'Elektrif' 'Infraforv' 'UHDist' 'TrSys'
 'TComr' 'Status' 'BdlSeq' 'geometry']


In [351]:
# load the dataset containg the information if a passenger can board a train
stations = gpd.read_file('./stations.gpkg')[['TplKod','PlNamn','Sign', 'SignTxt', 'PlTyp','Tillh_dpd','Tillh_dp_lp', 'Bandel', 'ResUtbyte']]
stations = stations[stations.PlTyp != 'dp*']

print('dataset size: ', stations.shape[0])
print('dataset columns: ', stations.columns.values)
# the column ResUtbyte is the one we are interested in below:

dataset size:  1611
dataset columns:  ['TplKod' 'PlNamn' 'Sign' 'SignTxt' 'PlTyp' 'Tillh_dpd' 'Tillh_dp_lp'
 'Bandel' 'ResUtbyte']


In [352]:
filtered_dataset = dataset.copy()

In [353]:
remove_list = stations[stations.ResUtbyte != 'j'].SignTxt.values
for to_remove in filtered_dataset[(filtered_dataset.PlSignFr.isin(remove_list))].PlSignFr.unique():

    station = stations[stations.SignTxt == to_remove] 

    matched = stations[stations.Tillh_dp_lp == station.PlNamn.values[0]]
    if matched.shape[0] != 0:
        if matched.ResUtbyte.values[0] == 'j':
            continue                   
        
    # find rows with station that should be removed
    pre_row = filtered_dataset[(filtered_dataset.PlSignTi == to_remove)]
    suc_row = filtered_dataset[(filtered_dataset.PlSignFr == to_remove)]

    filtered_dataset.loc[pre_row.index,'PlSignTi'] = suc_row.PlSignTi.values[0]
    filtered_dataset.loc[pre_row.index,'PlNamnTi'] = suc_row.PlNamnTi.values[0]

    filtered_dataset.drop(suc_row.index, inplace=True, axis=0)
filtered_dataset

,TxID,StrNr,Strak,BdlNr,Bandel,Forbind_1,PlSignFr,PlNamnFr,PlSignTi,PlNamnTi,AntalSpar,Elektrif,Infraforv,UHDist,TrSys,TComr,Status,BdlSeq,geometry
2,T230282,8.0,Norra stambanan,217.0,Bollnäs-Ljusdal,Ab-Vl,Ab,Arbrå,Vl,Vallsta,1.0,Ja,Trafikverket infrastruktur,Mitt,sysH,Gä,Öppen,217104.0,"LINESTRING (573518.164 6816036.145, 573517.398..."
8,T230195,99.0,Inlandsbanan,161.0,(Gällivare)-(Arvidsjaur),Ajr-Nkr,Ajr,Arvidsjaur,Nkr,Norra Kikkejaur,1.0,Nej,Inlandsbanan AB,IBAB,sysM,Gä,Öppen,161101.0,"LINESTRING (692144.289 7280445.156, 692142.397..."
9,T230031,21.0,Malmbanan,111.0,(Peuravaara)-Riksgränsen-(Björnfjell),Ak-Bln,Ak,Abisko östra,Bln,Björkliden,1.0,Ja,Trafikverket infrastruktur,Nord,sysH,Bdn,Öppen,111110.0,"LINESTRING (657624.433 7586639.317, 657620.394..."
11,T231056,74.0,(Alvhem) - Lilla Edet,666.0,(Alvhem)-Lilla Edet,Alh-Löö,Alh,Alvhem,Let,Lilla Edet,1.0,Nej,Trafikverket infrastruktur,Väst,sysS,G,Öppen,666101.0,"LINESTRING (332165.875 6433556.851, 331653.000..."
12,T230989,11.0,Norge-/Vänerbanan med Nordlänken,635.0,Öxnered-(Älvängen),Alh-Än,Alh,Alvhem,Än,Älvängen,2.0,Ja,Trafikverket infrastruktur,Väst,sysH,G,Öppen,635106.0,"LINESTRING (332165.875 6433556.851, 332165.685..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1449,T231204,24.0,Malmö närområde,901.0,Burlöv-Malmö C.Arlöv-Lockarp,Övn-Fsb,Övn,Östervärn,Sea,Svedala,2.0,Ja,Trafikverket infrastruktur,Syd,sysH,M,Öppen,901105.0,"LINESTRING (376125.298 6164076.776, 376258.000..."
1450,T231312,94.0,(Östervärn) - Brågarp,964.0,(Östervärn)-Brågarp,Övn-Sege,Övn,Östervärn,Båp,Brågarp,1.0,Nej,Trafikverket infrastruktur,Syd,None,M,Ejunderh,964101.0,"LINESTRING (376125.298 6164076.776, 376125.412..."
1452,T230984,11.0,Norge-/Vänerbanan med Nordlänken,635.0,Öxnered-(Älvängen),Öx-Thn,Öx,Öxnered,Thn,Trollhättan,2.0,Ja,Trafikverket infrastruktur,Väst,sysH,G,Öppen,635101.0,"LINESTRING (340504.707 6471923.075, 340458.000..."
1453,T231025,15.0,Älvsborgsbanan,652.0,(Öxnered)-(Håkantorp),Öx-Vg,Öx,Öxnered,Vg,Vänersborg central,1.0,Ja,Trafikverket infrastruktur,Väst,sysH,G,Öppen,652101.0,"LINESTRING (340504.707 6471923.075, 340597.519..."


In [354]:
df = filtered_dataset[['PlSignFr', 'PlSignTi']]
df

,PlSignFr,PlSignTi
2,Ab,Vl
8,Ajr,Nkr
9,Ak,Bln
11,Alh,Let
12,Alh,Än
...,...,...
1449,Övn,Sea
1450,Övn,Båp
1452,Öx,Thn
1453,Öx,Vg


In [355]:
df = pd.concat([df, pd.DataFrame(data=df[['PlSignTi', 'PlSignFr']].values, columns=['PlSignFr', 'PlSignTi'])])
df

,PlSignFr,PlSignTi
2,Ab,Vl
8,Ajr,Nkr
9,Ak,Bln
11,Alh,Let
12,Alh,Än
...,...,...
770,Sea,Övn
771,Båp,Övn
772,Thn,Öx
773,Vg,Öx


In [356]:
adj_matrix = pd.crosstab(df.PlSignFr, df.PlSignTi)
adj_matrix

PlSignTi,Ab,Aik,Ajr,Ak,Alh,Ar,Arb,Arnc,Arnn,Arns,...,Ökn,Önd,Ör,Örs,Ös,Öso,Öte,Övm,Övn,Öx
PlSignFr,,,,,,,,,,,,,,,,,,,,,
Ab,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Aik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ajr,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ak,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Alh,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Öso,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Öte,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Övm,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [338]:
adj_matrix_numpy = adj_matrix.to_numpy()

n = 1
result = np.linalg.matrix_power(adj_matrix_numpy , n)
print("grannmatrisen upphöjt till", n, "är:")
print(result)

# Räkna antalet icke-noll element i varje rad
non_zero_counts = np.count_nonzero(result, axis=1)

# Hämta indexen för de 10 raderna med flest icke-noll element
top_10_indices = np.argsort(non_zero_counts)[::-1][:10]

print("Top 10 rader med flest icke-noll element är:")
for index in top_10_indices:
    print("Rad", adj_matrix.columns[index], ":", result[index])
    print("Antal icke-noll element:", non_zero_counts[index])

grannmatrisen upphöjt till 1 är:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Top 10 rader med flest icke-noll element är:
Rad N : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0